In [1]:
import networkx as nx
from regraph import Rule, plot_rule


from regraph.neo4j.hierarchy import Neo4jHierarchy
import regraph.neo4j.cypher_utils as cypher


# initialize the neo4j driver, wrappped into Neo4jHierarchy object
h = Neo4jHierarchy(uri="bolt://localhost:7687", user="neo4j", password="admin")
h._clear()

nodes = [
    ("a", {"name": {"Jack"}, "age": {43}, "hobby": {"hiking", "music"}}), 
    ("b", {"name": {"Robert"}, "age": {35}, "hobby": {"sport", "music"}}),
    ("c", {"name":{ "Paul"}, "age": {18}}),
    ("d", {"name": {"Joe"}, "age": {12}}),
    ("e", {"name": {"Paul"}, "age": {18}})
]
edges = [
    ("a", "b", {"type": {"friends"}}), 
    ("a", "c", {"type": {"colleagues"}}),
    ("b", "c", {"type": {"colleagues"}}),
    ("d", "a", {"type": {"child_of"}}), 
    ("d", "e", {"type": {"enemies"}}),
    ("e", "e", {"type": {"likes"}})
]
h.add_graph('actionGraph', nodes, edges)


nodes = [
    ("a", {"type": {"Adult"}, "age": {43, 35, 18}, "name": {"Paul", "Jack", "Robert"}, "hobby": {"hiking", "music", "sport"}}),
    ("b", {"type": {"Child"}, "age": {12, 18}, "name": {"Paul", "Joe"}})
]
edges = [
    ("a", "a", {"type": {"friends", "colleagues"}}),
    ("b", "a", {"type": {"child_of"}}),
    ("b", "b", {"type": {"friends", "enemies"}})
]
h.add_graph('metaModel', nodes, edges)

mapping = {
    "a":"a",
    "b":"a",
    "c":"a",
    "d":"b",
    "e":"b"
}
h.add_typing('actionGraph', 'metaModel', mapping)

CREATE CONSTRAINT ON (n:actionGraph) ASSERT n.id IS UNIQUE
OPTIONAL MATCH (same_id_node:actionGraph) 
WHERE same_id_node.id = 'a' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['a' + same_id_node.count]
	ELSE ['a'] END AS new_id_a 
		CREATE (a:actionGraph { id : new_id_a }) 
SET a.name=['Jack']
SET a.age=[43]
SET a.hobby=['hiking', 'music']
WITH a, new_id_a OPTIONAL MATCH (same_id_node:actionGraph) 
WHERE same_id_node.id = 'b' 
FOREACH(new_count 
	IN CASE WHEN same_id_node IS NOT NULL
	THEN [coalesce(same_id_node.count, 0) + 1]
	ELSE [] END | 
		SET same_id_node.count = new_count) 
WITH same_id_node 
UNWIND
	CASE WHEN same_id_node IS NOT NULL
	THEN ['b' + same_id_node.count]
	ELSE ['b'] END AS new_id_b 
		CREATE (b:actionGraph { id : new_id_b }) 
SET b.name=['Robert']
SET b.age=[35]
SET b.hobby=['music', 'spo

In [4]:
pattern = nx.DiGraph()
pattern.add_node("child", type="Child")

rule = Rule.from_transform(pattern)
clone_name, _ = rule.inject_clone_node("child", "teenager")
rule.inject_remove_node_attrs(clone_name, {"type": "Child"})
rule.inject_add_node_attrs(clone_name, {"type": "Teenager"})

instance = {
    "child": "b"
}

p_typing = {
    "actionGraph": {
        "d": "child",
        "e": clone_name
    }
}

In [5]:
rule.removed_node_attrs()

{'teenager': {'type': {'Child'}}}

In [6]:
h.rewrite("metaModel", rule, instance, p_typing=p_typing)

CREATE CONSTRAINT ON (n:metaModel) ASSERT n.id IS UNIQUE
// Match nodes and edges of the instance 
MATCH (lhs_child:metaModel { id : 'b'})  


// Cloning node 'child' of the lhs 
// Create clone corresponding to 'teenager' of the preserved part
// create a node corresponding to the clone
CREATE (p_teenager:metaModel) 
WITH p_teenager, toString(id(p_teenager)) as p_teenager_id, lhs_child.id as original_old, lhs_child 
// set the id property of the original node to NULL
SET lhs_child.id = NULL
// copy all the properties of the original node to the clone
SET p_teenager = lhs_child
// set id property of the clone to neo4j-generated id
SET p_teenager.id = toString(id(p_teenager)), p_teenager.count = NULL
// set back the id property of the original node
SET lhs_child.id = original_old
WITH p_teenager, toString(id(p_teenager)) as p_teenager_id, lhs_child 
WITH [] as sucIgnore, [] as predIgnore, lhs_child, p_teenager, p_teenager_id 
// match successors and out-edges of a node to be cloned
OPTI

(<regraph.neo4j.hierarchy.Neo4jHierarchy at 0x7fea10c3db00>,
 {'child': 'b', 'teenager': '166438'})